In [152]:
# Ensure repo root is on sys.path and `data/` can be imported reliably
from pathlib import Path
import sys, os

# Locate project root by walking up until we find a `data/` directory
p = Path.cwd()
while not (p / 'data').exists() and p.parent != p:
    p = p.parent
repo_root = p

# Add repo root to sys.path so `import data` works
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

print(f"Repo root added to sys.path: {repo_root}")

# Use the correct class name: TSPLIBLoader (capital B)
from data.tsplib_loader import TSPLIBLoader
loader = TSPLIBLoader(data_dir=str(repo_root / "data" / "problem_instances"))
print("TSPLIBLoader imported and loader created (data/problem_instances)")

Repo root added to sys.path: /Users/connorplaks/rl-ga-travelin-salesman
TSPLIBLoader imported and loader created (data/problem_instances)


In [ ]:
import os

from data.tsplib_loader import TSPLIBLoader
loader = TSPLIBLoader(data_dir="../../data/problem_instances")
instances = ['eil51', 'berlin52', 'st70', 'eil76', 'kroA100', 'pr107']
instances = ['eil51', 'berlin52', 'st70', 'eil76', 'kroA100', 'pr107']
test_instance_idx = 0
instance = instances[test_instance_idx]
print(f"Loading instance: {instance}")
# Load instance
data = loader.load_instance('eil51')

# Load optimal tour
tour = loader.load_optimal_tour(instance)

Loading instance: eil51


In [154]:
data

{'name': 'eil51',
 'dimension': 51,
 'coordinates': {1: [37, 52],
  2: [49, 49],
  3: [52, 64],
  4: [20, 26],
  5: [40, 30],
  6: [21, 47],
  7: [17, 63],
  8: [31, 62],
  9: [52, 33],
  10: [51, 21],
  11: [42, 41],
  12: [31, 32],
  13: [5, 25],
  14: [12, 42],
  15: [36, 16],
  16: [52, 41],
  17: [27, 23],
  18: [17, 33],
  19: [13, 13],
  20: [57, 58],
  21: [62, 42],
  22: [42, 57],
  23: [16, 57],
  24: [8, 52],
  25: [7, 38],
  26: [27, 68],
  27: [30, 48],
  28: [43, 67],
  29: [58, 48],
  30: [58, 27],
  31: [37, 69],
  32: [38, 46],
  33: [46, 10],
  34: [61, 33],
  35: [62, 63],
  36: [63, 69],
  37: [32, 22],
  38: [45, 35],
  39: [59, 15],
  40: [5, 6],
  41: [10, 17],
  42: [21, 10],
  43: [5, 64],
  44: [30, 15],
  45: [39, 10],
  46: [32, 39],
  47: [25, 32],
  48: [25, 55],
  49: [48, 28],
  50: [56, 37],
  51: [30, 40]},
 'distance_matrix': array([[ 0., 12., 19., ..., 26., 24., 14.],
        [12.,  0., 15., ..., 21., 14., 21.],
        [19., 15.,  0., ..., 36., 27.,

In [329]:
import numpy as np
from data.tsplib_loader import TSPLIBLoader

class TSPRLAgent:
    """Reinforcement learning agent for TSP route construction."""
    
    def __init__(self, config):
        # extract the test scenerio data
        loader = TSPLIBLoader(data_dir = "../../data/problem_instances")
        instance = config['instance']
        data = loader.load_instance(instance)
        self.test_data = data
        self.distance_matrix = data['distance_matrix']
        self.name = data['name']
        self.dimension = data['dimension'] 
        self.coordinates = data['coordinates']
        self.optimal_value = data['optimal_value']
        
        # pull out the RL hyper parameters
        self.config = config
        self.alpha = config.get('alpha', 0.01)  # Learning rate
        self.gamma = config.get('gamma', 0.15)  # Discount factor
        self.episodes = config.get('episodes', 1000)
        self.reward_type = config.get('reward_type', 1)
        self.epsilon_greedy_type = config.get('epsilon_greedy_type',1)

        # initialize our Q tables
        self.QA = np.zeros([self.dimension, self.dimension]) # Q-value table
        self.QB = np.zeros([self.dimension, self.dimension]) # Q-value table
        self.done = False
        
        # # set random seed
        # random_seed = 42
        # np.random.seed(random_seed)
    
    def action_space(self, visited):
        """Return the action space (list of cities that have not been visited)."""
        return [coord for coord in range(self.dimension) if coord not in visited]
    def reward(self, current_city, action):
        """Calculate reward for moving from current_city to next_city."""
        distance = self.distance_matrix[current_city, action]
        
        # return the reward type specified in the config (either inverse of distance or negative squared distance)
        if self.reward_type == 1:
            return 1 / distance
        if self.reward_type == 2:
            return -distance**2
            
    def step(self, current_city, action, visited):
        """Take a step in the environment."""
        
        # compute the reward at our current state and action
        reward = self.reward(current_city, action)
        
        # define the next state
        next_city = action
        
        # compute our list of visited states at from the action
        next_visited = visited | {action}
        
        # if we are at the last city, return to the initial city
        if len(next_visited) == self.dimension:
            # reward += self.reward(next_city, 0)
            self.done= True
        return next_city, reward, next_visited
    def epsilon_greedy_policy(self, epsilon, current_city, visited):
        """Select next city using epsilon-greedy policy."""
        
        # compute the action space of remaining cities from the ones we've visited
        action_space = self.action_space(visited)
        
        if np.random.rand() < epsilon:
            # Explore - choose a random city
            next_city = np.random.choice(action_space)
        else:
            # Exploit - choose the best known city
            next_city = max(action_space, key = lambda a: 0.5* (self.QA[current_city, a] + self.QB[current_city, a]))
        return next_city
    def compute_epsilon(self, episode):
        """Compute our epsilon for epsilon greedy double Q-learning"""
        
        if self.epsilon_greedy_type == 1:
            return 1 - episode / self.episodes
        if self.epsilon_greedy_type == 2:
            return 1 - (episode / self.episodes)**6
        else:
            1 - 0.1 * (np.floor(episode/self.episodes))
        
    def train_episode(self, epsiode_num):
        """Generate RL epsiode the RL agent."""
        
        # initialize environment
        current_city = 0
        visited = frozenset({current_city})
        self.done = False
        epsilon = self.compute_epsilon(epsiode_num)
        
        # initialize Q value for all final states
        for state in range(1,self.dimension):
            self.QA[state, 0] = self.reward(state, 0)
            self.QB[state, 0] = self.reward(state, 0)
        
        # loop until episode is done
        while not self.done:
            
            # pick action using epsilon-greedy policy
            action = self.epsilon_greedy_policy(epsilon, current_city, visited)
            
            # proceed to next state
            next_city, reward, next_visited = self.step(current_city, action, visited)
            
            # update Q-values
            if len(next_visited) == self.dimension:
                
                random_choice = np.random.rand()
                # if random_choice < 0.5:
                self.QA[(current_city, action)] = self.QA[current_city, action] + self.alpha * (reward + self.gamma * (self.QB[next_city, 0]) - self.QA[current_city, action])
                # else:
                self.QB[(current_city, action)] = self.QB[current_city, action] + self.alpha * (reward + self.gamma * (self.QA[next_city, 0]) - self.QB[current_city, action])
                self.done = True
            else:
                random_choice = np.random.rand()
                # if random_choice < 0.5:
                self.QA[(current_city, action)] = self.QA[current_city, action] + self.alpha * (reward + self.gamma * max(self.QB[next_city, a] for a in self.action_space(next_visited)) - self.QA[current_city, action])
                # else:
                self.QB[(current_city, action)] = self.QB[current_city, action] + self.alpha * (reward + self.gamma * max(self.QA[next_city, a] for a in self.action_space(next_visited)) - self.QB[current_city, action])
            
            # increment state
            current_city = next_city
            visited = next_visited

            
    def train(self):
        """Train the RL agent."""
        
        # perform Q-learning for the number of episodes specified
        for episode in range(self.episodes):
            self.train_episode(episode)
    
    
    def optimal_route(self):
        """Construct optimal route using trained policy."""
        
        # initialize enviorment
        current_city = 0
        visited = frozenset({current_city})
        route = [current_city]
        self.done = False
        cost = 0
        
        # Loop through the states selecting the action that maximizes our learned Q functions
        while not self.done:
            action_space = self.action_space(visited)
            next_city = max(action_space, key = lambda a: 0.5* (self.QA[current_city, a] + self.QB[current_city, a]))
            route.append(next_city)
            visited = visited | {next_city}
            cost += self.distance_matrix[current_city, next_city]
            current_city = next_city
            if len(visited) == self.dimension:
                route.append(0)  # Return to starting city
                cost += self.distance_matrix[next_city, 0]
                self.done = True
                
        return cost,route
        


In [332]:

instances = ['eil51', 'berlin52', 'st70', 'eil76', 'kroA100', 'pr107']
instance = 'berlin52'

agent_config = {
    'alpha': 0.01,
    'gamma': 0.01,
    'episodes': 10000,
    'instance' : instance,
    'reward_type' : 1,
    'epsilon_greedy_type' : 1
}
agent = TSPRLAgent(agent_config)
agent.train()
cost, constructed_route = agent.optimal_route()
print("Cost:", cost)



Cost: 8862.0


In [285]:
cost, constructed_route = agent.optimal_route()
print("Cost:", cost)
print("Constructed route:", constructed_route)
sorted(constructed_route)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 26, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24,

[0,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50]

In [ ]:
print(tour)

[1, 22, 8, 26, 31, 28, 3, 36, 35, 20, 2, 29, 21, 16, 50, 34, 30, 9, 49, 10, 39, 33, 45, 15, 44, 42, 40, 19, 41, 13, 25, 14, 24, 43, 7, 23, 48, 6, 27, 51, 46, 12, 47, 18, 4, 17, 37, 5, 38, 11, 32]


In [ ]:
tour = loader.load_optimal_tour(instance)
data = loader.load_instance(instance)
instances = ['eil51', 'berlin52', 'st70', 'eil76', 'kroA100', 'pr107']

cost = 0
for i in range(len(tour)-1):
    cost += data['distance_matrix'][tour[i]-1, tour[i+1]-1]
cost += data['distance_matrix'][tour[-1]-1, tour[0]-1]
cost

426.0

In [328]:
print(agent.QA[:,0])

[0.         0.08333333 0.05263158 0.03225806 0.04545455 0.05882353
 0.04347826 0.08333333 0.04166667 0.02941176 0.08333333 0.04761905
 0.02380952 0.03703704 0.02777778 0.05263158 0.03225806 0.03571429
 0.02173913 0.04761905 0.03703704 0.14285714 0.04545455 0.03448276
 0.03030303 0.05263158 0.125      0.0625     0.04761905 0.03030303
 0.05882353 0.16666667 0.02325581 0.03225806 0.03703704 0.03225806
 0.03333333 0.05263158 0.02325581 0.01785714 0.02272727 0.02222222
 0.02941176 0.02631579 0.02380952 0.07142857 0.04347826 0.08333333
 0.03846154 0.04166667 0.07142857]


In [38]:
frozen_set = frozenset({0, 1, 2})
frozen_set = frozen_set | {3}
test = {}
test[(0, frozen_set, 1)] = "test_value"

In [18]:
import numpy as np
D = np.random.randint(1, 20, size=(5, 5)).astype(float)
np.fill_diagonal(D, 0)

# Ensure asymmetry
for i in range(5):
    for j in range(i+1, 5):
        D[i, j] += np.random.randint(0, 5)
D

array([[ 0.,  6.,  8., 10., 10.],
       [ 4.,  0.,  4., 11., 19.],
       [13., 16.,  0., 18.,  5.],
       [11., 13.,  8.,  0., 17.],
       [10., 12.,  4., 19.,  0.]])

In [20]:
visited_mask = 1 << 0
visited_mask | (1 << 5)

33

In [ ]:
Instead of using the 